In [7]:
import mlflow
import pandas as pd

# Set the tracking URI to your DagsHub MLflow instance
mlflow.set_tracking_uri("https://dagshub.com/HassanBarka/MLOps.mlflow") 
reports_path = "./models/run_info.json"
with open(reports_path, 'r') as file:
    run_info = json.load(file)

model_name = run_info['model_name']  



try:
    # Create an MlflowClient to interact with the MLflow server
    client = mlflow.tracking.MlflowClient()

    # Get the latest version of the model in the Production stage
    versions = client.get_latest_versions(model_name, stages=["Production"])

    if versions:
        latest_version = versions[0].version
        run_id = versions[0].run_id  # Fetching the run ID from the latest version
        print(f"Latest version in Production: {latest_version}, Run ID: {run_id}")

        # Construct the logged_model string
        logged_model = f'runs:/{run_id}/{model_name}'
        print("Logged Model:", logged_model)

        # Load the model using the logged_model variable
        loaded_model = mlflow.pyfunc.load_model(logged_model)
        print(f"Model loaded from {logged_model}")

        return lo
    else:
        print("No model found in the 'Production' stage.")

except Exception as e:
    print(f"Error fetching model: {e}")

/tmp/ipykernel_58832/824515244.py:19: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  versions = client.get_latest_versions(model_name, stages=["Production"])


Latest version in Production: 4, Run ID: 6c6a46cd33d14bf2aef430592b04cdd3
Logged Model: runs:/6c6a46cd33d14bf2aef430592b04cdd3/DT
Model loaded from runs:/6c6a46cd33d14bf2aef430592b04cdd3/DT


In [21]:
from backend.example_json.drought_info import DroughtModel
from backend.preprocessing import scale_data,date_processing

In [12]:
example_data = DroughtModel.Config.json_schema_extra["example"]
drought_instance = DroughtModel(**example_data)
drought_instance

DroughtModel(fips=1001, date=datetime.datetime(2000, 1, 4, 0, 0), PRECTOT=15.95, PS=100.29, QV2M=6.42, T2M=11.4, T2MDEW=6.09, T2MWET=6.1, T2M_MAX=18.09, T2M_MIN=2.16, T2M_RANGE=15.92, TS=11.31, WS10M=3.84, WS10M_MAX=5.67, WS10M_MIN=2.08, WS10M_RANGE=3.59, WS50M=6.73, WS50M_MAX=9.31, WS50M_MIN=3.74, WS50M_RANGE=5.58)

In [68]:
data= pd.DataFrame(drought_instance.dict(),index=[0])
data

/tmp/ipykernel_58832/761339514.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  data= pd.DataFrame(drought_instance.dict(),index=[0])


,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M_RANGE,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE
0,1001,2000-01-04,15.95,100.29,6.42,11.4,6.09,6.1,18.09,2.16,15.92,11.31,3.84,5.67,2.08,3.59,6.73,9.31,3.74,5.58


In [69]:
import pickle
data = date_processing(data)

# data = scale_data(data)
with open('models/standard_scaler.pkl', 'rb') as file:
        scalar = pickle.load(file)
df = scalar.transform(data)
df

array([[-1.99753287,  3.73781825,  0.68693002, -0.32331778, -0.18481095,
        -0.13210458, -0.13497068, -0.11167176, -0.56027147,  1.13414046,
        -0.19737153,  0.31112926,  0.35949965,  0.22547351,  0.32953397,
         0.71878266,  0.76680554,  0.39439555,  0.58362176, -1.54526718,
        -1.62195655, -1.33240328]])

In [56]:
# Colonnes attendues par le modèle
columns = ['fips', 'PRECTOT', 'PS', 'QV2M', 'T2M', 'T2MDEW', 'T2MWET',
           'T2M_MAX', 'T2M_MIN', 'T2M_RANGE', 'TS', 'WS10M', 'WS10M_MAX',
           'WS10M_MIN', 'WS10M_RANGE', 'WS50M', 'WS50M_MAX', 'WS50M_MIN',
           'WS50M_RANGE', 'year', 'month', 'day']

# Conversion en DataFrame
df = pd.DataFrame(df, columns=columns)
loaded_model.predict(df)

array([1])

In [73]:
df = pd.read_csv("/home/hababi/data/drought.csv")

In [81]:
df = df[pd.DatetimeIndex(df['date']).year >= 2008]
df_random_sample = df.sample(n=10, random_state=0)  # random_state pour reproductibilité



In [84]:
df_random_sample

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
287316,12067,2015-08-18,7.34,101.47,18.85,27.54,24.29,24.29,30.84,24.58,...,27.82,1.55,2.00,0.97,1.03,2.20,3.00,1.41,1.59,0.0023
1069430,26017,2011-05-31,3.72,99.39,14.42,21.86,19.59,19.59,26.29,17.03,...,19.96,6.42,8.97,3.51,5.46,9.63,13.92,5.42,8.50,0.0000
2203280,47179,2016-06-21,7.66,93.67,12.92,23.02,16.85,16.85,30.14,16.94,...,23.14,2.39,3.29,1.48,1.80,4.91,6.19,2.65,3.54,1.0000
1572294,35021,2010-03-02,0.00,85.57,3.23,5.24,-4.25,-4.18,14.13,-3.01,...,6.32,2.71,3.53,1.45,2.08,4.28,6.59,1.95,4.64,0.0000
2246754,48087,2016-09-06,1.54,94.42,15.24,27.66,19.68,19.68,33.28,21.89,...,28.33,6.50,8.13,3.95,4.18,8.98,10.89,6.33,4.56,0.0000
1745321,38023,2011-05-10,31.42,92.99,5.60,5.73,4.24,4.24,8.89,1.58,...,6.04,9.11,11.35,6.79,4.56,11.27,13.78,9.02,4.77,0.0000
262123,12009,2008-10-14,0.61,101.77,15.14,25.04,20.79,20.79,26.97,23.18,...,24.66,6.85,8.00,5.21,2.79,9.46,10.86,8.35,2.51,0.0000
815713,20131,2010-09-28,0.00,96.75,8.77,17.40,11.47,11.47,23.46,11.53,...,16.81,4.33,6.37,1.44,4.94,6.18,10.19,2.07,8.12,0.0000
115178,5095,2014-06-24,5.81,100.65,15.75,25.59,21.22,21.22,30.10,22.48,...,25.57,3.93,5.58,2.14,3.44,5.63,7.42,3.95,3.47,0.0000
1897099,40083,2013-04-16,0.05,97.49,7.95,13.45,9.72,9.72,19.46,8.85,...,14.81,4.02,5.47,2.09,3.38,4.62,6.89,2.90,3.99,2.0000


In [85]:
df_random_sample.drop(columns=['score'],inplace=True)

In [86]:
# Sauvegarder les données dans un nouveau fichier CSV
df_random_sample.to_csv("./backend/exemple_df.csv", index=False)